https://adventofcode.com/2023/day/21

In [32]:
with open("i21.txt") as f:
    lines = [line.strip() for line in f.readlines()]
    
for i, line in enumerate(lines):
    for j, c in enumerate(line):
        if c =="S":
            ix,iy = i,j
size=len(lines)
ix,iy,size

(65, 65, 131)

In [33]:
rims = []
garden = [[t for t in line] for line in lines]
def step(x,y,garden=garden):
    if garden[x][y] != "#":
        garden[x][y] = "X"

def show(garden=garden):
    for line in garden:
        print(''.join(line))
        
for row in range(66):    
    for col in range(66):
        step(row+col,65+col-row)


show()

.................................................................X.................................................................
.##.....#........#........#...........#......#....##.....##.....X.X............#...#.......#.....................#......#.#........
...........................#.......#...#.#...#....#............X.X.X..........#................##.......#....................#.#...
..............#..............##...#.....#.#...#..#.......#....X.X.X.X........................#..#......#.......#....#...........##.
..........##...#..#......#...#.#.............................X.X.X.X.X....#....................#.....................#.....#.......
..#............##.....................##.........#..........X.X.X.X.X.X.........#.............##.........##..................#.....
..............#.#..............................#...........X.X.X.X.X.X.X.......#....#...#..#.......................#...##..........
.....................#...#....#.......#.............#.#...X.X.X.X.##X.X.X...

In [37]:
reachables = set((65,65)) # x,y,steps
dirs = [(-1,0),(0,1),(1,0),(0,-1)]
workout_area = [line.copy() for line in garden]
step_map = {0:[(65,65)]}

for i in range(64):
    tiles_to_check = step_map[i]
    step_map[i+1] = []
    for x,y in tiles_to_check:
        for dx,dy in dirs:
            nx, ny = x+dx, y+dy
            if not (nx,ny) in reachables and workout_area[nx][ny] != "#":
                reachables.add((nx,ny))
                step_map[i+1].append((nx,ny))
                if i%2:
                    workout_area[nx][ny] = "o"
                else: workout_area[nx][ny] = "_"


print(sum(sum(1 for o in line if o == "o") for line in workout_area))
show(workout_area)


3830
.................................................................X.................................................................
.##.....#........#........#...........#......#....##.....##.....XoX............#...#.......#.....................#......#.#........
...........................#.......#...#.#...#....#............Xo_oX..........#................##.......#....................#.#...
..............#..............##...#.....#.#...#..#.......#....Xo_o_oX........................#..#......#.......#....#...........##.
..........##...#..#......#...#.#.............................Xo_o_o_oX....#....................#.....................#.....#.......
..#............##.....................##.........#..........Xo_o_o_o_oX.........#.............##.........##..................#.....
..............#.#..............................#...........Xo_o_o_o_o_oX.......#....#...#..#.......................#...##..........
.....................#...#....#.......#.............#.#...Xo_o_o_o##_o_

## part 2
ok now 26501365 steps

unsurprisingly, brute force does work. (200steps already take 11s) so what do we know?
- intially, i thought we could just assume that all full 64 steps borders can be reached. meaning a parallellogram. then it would be easy to calc the area, take half (all even steps smaller than max_step can also be reached), subtract #s, done
- but thats not the case. there are some steps that are blocked. they are blocked by the grid itself.
 
what about the border and the periodicity?
- if we know the min steps to the borders, what to do with that?

what about the 26 mill?
- suggests that only the number of steps should be kept, not all tiles themselves , as it will be a very large number of steps

so we need to adjust our approach - posibilities
- tree search: grows exponentially
- area approach: calc theoretic max area, how to find out unreachable areas?

distance approach: 
- calc distances to each border and center, 
- the border is unobstructed, 
so once the min distance between border and center is know
-  go through garden patches, like through tiles but with some speed ups:
  - the whole patch only needs to be checked if the max distance for any tile to that edge > max steps 
  - so from the 200_000 patches, only the outermost need to be checked

importantly, the border steps are all homogeneous and minimum.
i suspect this means that 


In [80]:

to_visited = {(65,65):0} # x,y,steps

dirs = [(-1,0),(0,1),(1,0),(0,-1)]
workout_area = [line.copy() for line in garden]

for i in range(160):
    for x,y in [k for k, v in to_visited.items() if v ==i]:
        for dx,dy in dirs:
            nx, ny = x+dx, y+dy
            if any([nx<0,ny<0,nx==size, ny==size]):continue
            if not (nx,ny) in to_visited and workout_area[nx][ny] != "#":
                to_visited[(nx,ny)]=i+1
                step_map[i+1].append((nx,ny, i+1))

left = {(x,y):s for (x,y), s in to_visited.items() if y==0}
top = {(x,y):s for (x,y), s in to_visited.items() if x==0}
right = {(x,y):s for (x,y), s in to_visited.items() if y==size-1}
bottom = {(x,y):s for (x,y), s in to_visited.items() if x==size-1}
patch_dirs = [top, right, bottom, left]


{(0, 65): 65,
 (0, 66): 66,
 (0, 64): 66,
 (0, 67): 67,
 (0, 63): 67,
 (0, 68): 68,
 (0, 62): 68,
 (0, 69): 69,
 (0, 61): 69,
 (0, 70): 70,
 (0, 60): 70,
 (0, 71): 71,
 (0, 59): 71,
 (0, 72): 72,
 (0, 58): 72,
 (0, 73): 73,
 (0, 57): 73,
 (0, 74): 74,
 (0, 56): 74,
 (0, 75): 75,
 (0, 55): 75,
 (0, 76): 76,
 (0, 54): 76,
 (0, 77): 77,
 (0, 53): 77,
 (0, 78): 78,
 (0, 52): 78,
 (0, 79): 79,
 (0, 51): 79,
 (0, 80): 80,
 (0, 50): 80,
 (0, 81): 81,
 (0, 49): 81,
 (0, 82): 82,
 (0, 48): 82,
 (0, 83): 83,
 (0, 47): 83,
 (0, 84): 84,
 (0, 46): 84,
 (0, 85): 85,
 (0, 45): 85,
 (0, 86): 86,
 (0, 44): 86,
 (0, 87): 87,
 (0, 43): 87,
 (0, 88): 88,
 (0, 42): 88,
 (0, 89): 89,
 (0, 41): 89,
 (0, 90): 90,
 (0, 40): 90,
 (0, 91): 91,
 (0, 39): 91,
 (0, 92): 92,
 (0, 38): 92,
 (0, 93): 93,
 (0, 37): 93,
 (0, 94): 94,
 (0, 36): 94,
 (0, 95): 95,
 (0, 35): 95,
 (0, 96): 96,
 (0, 34): 96,
 (0, 97): 97,
 (0, 33): 97,
 (0, 98): 98,
 (0, 32): 98,
 (0, 99): 99,
 (0, 31): 99,
 (0, 100): 100,
 (0, 30): 100,
 (0

In [ ]:
26501365/130

(203856.65384615384, 'patches')